In [1]:
import numpy as np
import pandas as pd

# from generative_example import get_real_data_preprocessed
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from scipy.stats import cauchy
import random


In [16]:
"""Generate syntheic data with the distribution of real data."""
print("Getting real data")
# X, y = get_real_data_preprocessed()
data = pd.read_csv("./no_stops_reviews.csv")
X = data["review"].values.astype("U")
y = data["label"]
vectorizer = CountVectorizer()
vectorizer.fit(X)
X_v = vectorizer.transform(X)
X_train, X_test, y_train, y_test = train_test_split(
    X_v, y, test_size=0.2, random_state=67575
)

print("Ingestion of real data done")

# model
model = MultinomialNB()
model.fit(X_train, y_train)

num_reviews_each = 25000

print(f"Generating {num_reviews_each} reviews for each class")

# decide length by "review" population distribution
# we used fitter package to get the best fit distribution
location = 244.3346105918917
scale = 93.64089186697534
r = cauchy.rvs(
    loc=location, scale=scale, size=num_reviews_each * 2, random_state=3254
)

length = []
for i in r:
    if i < 3:
        length.append(3)
        # print(i)
    elif i > 3910:
        length.append(3910)
        # print(i)
        # print(f"{length[-1]} is what we want")
    else:
        length.append(int(i))

Xg_pos = []
Xg_neg = []
s_pos = []
s_neg = []
Xg = []
yg = []

vocab = vectorizer.get_feature_names_out()
word_prob = np.exp(model.feature_log_prob_)

for i in range(num_reviews_each):
    # for postitive words
    # print(i)
    pos_word = random.choices(vocab, weights= word_prob[1], k=length[i])
    # print(length[i])
    # stich the words together to form a sentence
    # pos = list(pos_word)
    s_pos = " ".join(pos_word)
    Xg_pos.append(s_pos)
    # for negative words
    neg_word = random.choices(vocab, weights= word_prob[0], k=length[i])
    # stich the words together to form a sentence
    # neg = list(neg_word)
    s_neg = " ".join(neg_word)
    Xg_neg.append(s_neg)

yg_pos = np.ones(num_reviews_each)
pos_df = pd.DataFrame({"review": Xg_pos, "label": yg_pos})
yg_neg = np.zeros(num_reviews_each)
neg_df = pd.DataFrame({"review": Xg_neg, "label": yg_neg})


Getting real data
Ingestion of real data done
Generating 25000 reviews for each class


In [15]:
pos_df.head()

,review,label
0,get early never,1.0
1,exactly made almost,1.0
2,simple thoroughly humble,1.0
3,gets making seriously,1.0
4,long ending keep,1.0


In [18]:
# pos_df["review"].str.split().len().max()
pos_df["review"].str.split().apply(len).max()

3910

In [8]:
type(pos_word)

list